In [1]:
# importing required libraries
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

In [2]:
def detect_and_predict_mask(frame, face_net, mask_model):
    """
    This function detects faces in an image frame and predicts whether each face is masked or not.

    Args:
        frame: The image frame as a NumPy array.
        face_net: The pre-trained face detection model.
        mask_model: The trained mask detection model.

    Returns:
        A tuple containing:
            - locs: A list of bounding boxes for detected faces (startX, startY, endX, endY).
            - preds: A NumPy array containing the predicted probabilities of wearing a mask for each face.
    """

    (h, w) = frame.shape[:2]  # Extract frame height and width

    # Preprocess the frame for face detection
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_net.setInput(blob)
    detections = face_net.forward()

    faces = []
    locs = []
    preds = []

    # Loop through detected objects
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter detections based on confidence threshold (adjusted to 0.3 for potentially lower quality images)
        if confidence > 0.3:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Enforce box coordinates within frame boundaries
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # Extract face region from the frame
            face = frame[startY:endY, startX:endX]

            # Handle empty face region detection
            if face.size == 0:
                print(f"Warning: Empty face region detected at {startX, startY, endX, endY}")
                continue

            # Preprocess the extracted face image for mask prediction
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # Process detected faces (if any)
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")  # Convert faces list to NumPy array
        preds = mask_model.predict(faces, batch_size=32)  # Predict mask probabilities

        # Handle potential single-output format from the model (adjust to binary classification)
        if preds.shape[-1] == 1:
            preds = np.hstack([1 - preds, preds])  # Concatenate for binary (masked/not masked)

    else:
        print("No faces detected in the frame")

    # Print informative messages
    print(f"Number of faces detected: {len(faces)}")
    print(f"Shape of preds: {preds.shape if len(preds) > 0 else 'empty'}")

    return (locs, preds)


In [ ]:
# Initialize video capture object using default camera (index 0)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Loop continuously until user exits
while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Check if frame capture was successful
    if not ret:
        print("Error: Can't receive frame. Exiting ...")
        break

    # Resize the frame (optional, adjust dimensions as needed)
    try:
        frame = cv2.resize(frame, (800, 600))
    except cv2.error as e:
        print(f"Error resizing frame: {str(e)}")
        continue

    mask_model = tf.keras.models.load_model("mask_detector.keras")

    # Process the frame for face detection and mask prediction
    try:
        locs, preds = detect_and_predict_mask(frame, face_net, mask_model)

        # Handle the case where no faces were detected
        if len(locs) == 0:
            cv2.putText(frame, "No faces detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Process each detected face
        else:
            for (box, pred) in zip(locs, preds):
                (startX, startY, endX, endY) = box
                (mask, withoutMask) = pred

                # Determine mask/no mask label and color based on prediction
                label = "Mask" if mask > withoutMask else "No Mask"
                color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

                # Format label with predicted probability
                label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

                # Draw bounding box and label on the frame
                cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
                cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

        # Display the processed frame
        cv2.imshow("Frame", frame)

    # Catches any unexpected errors during processing and prints the traceback
    except Exception as e:
        print(f"Error processing frame: {str(e)}")
        traceback.print_exc()  # This will print the full traceback for debugging

    # Check for user input to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()
